In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import tensorflow as tf
from tensorflow.keras import models, layers, activations, initializers, metrics, utils, losses, optimizers

from sklearn import model_selection

In [2]:
# # colab on google drive 

# from google.colab import drive
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/groom/semi-3/fifa_pay_prediction/03_work_code/ukseongchoi'

In [3]:
train = pd.read_csv("../../00_original_data/FIFA_train.csv")
test = pd.read_csv("../../00_original_data/FIFA_test.csv")

submission = pd.read_csv("../../00_original_data/submission.csv")


In [4]:
# 필요하지 않는 feature 값 제거
train.drop(['id', 'name'], axis = 1, inplace = True)
test.drop(['id', 'name'], axis = 1, inplace = True)

In [5]:
#타겟 데이터
y_train = train['value']

#타겟 생성 후 필요없는 열제거
del train['value']

#학습 데이터
x_train = train.copy()

In [6]:
print(x_train['continent'].unique())
print(x_train['position'].unique())
print(x_train['prefer_foot'].unique())

['south america' 'europe' 'africa' 'asia' 'oceania']
['ST' 'GK' 'DF' 'MF']
['left' 'right']


In [7]:
# categorical data를 담고있는 Series는 특수 메서드인 cat 속성이 있다.
# 이를 통해 catagories 와 codes 나 categorical 메서드 등에 쉽게 접근할 수 있다.
# 하지만 시리즈에 바로 categories나 codes를 사용하면 에러가 발생하는데 이때 이용하는 것이 cat 속성이다.

x_train['continent']=x_train['continent'].astype('category').cat.codes
test['continent'] = test['continent'].astype('category').cat.codes

x_train['position'] = x_train['position'].astype('category').cat.codes
test['position'] = test['position'].astype('category').cat.codes

x_train['prefer_foot'] = x_train['prefer_foot'].astype('category').cat.codes
test['prefer_foot'] = test['prefer_foot'].astype('category').cat.codes


In [8]:
# 계약 연도를 object -> int 로 변환
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:] # 계약연도가 제일 끝에 있거나 단독으로 쓰이고 있어 뒤에서 4번째부터 가지고 옴.
    return int(string)

x_train['contract_until'] = x_train['contract_until'].apply(func)
test['contract_until'] = test['contract_until'].apply(func)

In [9]:
print(y_train)
x_train
# test

0       110500000.0
1        72000000.0
2        80000000.0
3        51000000.0
4        68000000.0
           ...     
8927        60000.0
8928        40000.0
8929        50000.0
8930        50000.0
8931        60000.0
Name: value, Length: 8932, dtype: float64


,age,continent,contract_until,position,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves
0,31,4,2021,3,0,5.0,94,94,4.0
1,27,2,2020,1,1,4.0,91,93,1.0
2,31,4,2021,3,1,5.0,91,91,3.0
3,32,2,2020,0,1,4.0,91,91,3.0
4,25,2,2021,1,1,3.0,90,93,1.0
...,...,...,...,...,...,...,...,...,...
8927,18,0,2019,2,1,1.0,48,63,3.0
8928,19,2,2020,0,1,1.0,47,59,2.0
8929,18,4,2021,0,1,1.0,47,64,2.0
8930,18,2,2021,1,1,1.0,47,65,1.0


In [10]:
#조합찾기 

from itertools import combinations 

arr = ['reputation' , 'stat_skill_moves']
categorics = []
for i in range(len(arr)+1):
  categorics = categorics + list(combinations(arr,i))

for k in range(len(categorics)):
  categorics[k] = list(categorics[k])

numerics = []
for i in range(len(categorics)):
  remove_set = list(categorics[i])
  numerics.append([i for i in arr if i not in remove_set])

categorical_base = ['continent', 'position']
numerical_base = ['stat_overall', 'stat_potential','contract_until', 'age']

for k in range(len(categorics)):
  categorics[k] = categorical_base + categorics[k]
  numerics[k] = numerical_base + numerics[k]

In [11]:
new_df = x_train.copy()
pd_list = []
for j in range(len(categorics)):
  new_df = x_train.copy()

  for categori_name in categorics[j]:


    num = len(new_df[categori_name].unique())
    make_name =[]
    for i in range(num):
      
      make_name.append(categori_name + str(i))

    en_df = pd.get_dummies(new_df[categori_name])

    en_df.columns = make_name


    del new_df[categori_name]

    new_df = pd.concat([new_df , en_df], axis=1 )

  pd_list.append(new_df)

In [12]:
a='''
카테고리로 해석할 열들에 대한 info 입니다.
pd_list[0] == {}
pd_list[1] == {}
pd_list[2] == {}
pd_list[3] == {}
'''.format(categorics[0],categorics[1],categorics[2],categorics[3])

print(a)


카테고리로 해석할 열들에 대한 info 입니다.
pd_list[0] == ['continent', 'position']
pd_list[1] == ['continent', 'position', 'reputation']
pd_list[2] == ['continent', 'position', 'stat_skill_moves']
pd_list[3] == ['continent', 'position', 'reputation', 'stat_skill_moves']



In [13]:
# for i in range(4):
#     del pd_list[0]['prefer_foot']
#     del pd_list[1]['prefer_foot']
#     del pd_list[2]['prefer_foot']
#     del pd_list[3]['prefer_foot']

In [14]:
x_train = pd_list[0]
x_train

,age,contract_until,prefer_foot,reputation,stat_overall,stat_potential,stat_skill_moves,continent0,continent1,continent2,continent3,continent4,position0,position1,position2,position3
0,31,2021,0,5.0,94,94,4.0,0,0,0,0,1,0,0,0,1
1,27,2020,1,4.0,91,93,1.0,0,0,1,0,0,0,1,0,0
2,31,2021,1,5.0,91,91,3.0,0,0,0,0,1,0,0,0,1
3,32,2020,1,4.0,91,91,3.0,0,0,1,0,0,1,0,0,0
4,25,2021,1,3.0,90,93,1.0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8927,18,2019,1,1.0,48,63,3.0,1,0,0,0,0,0,0,1,0
8928,19,2020,1,1.0,47,59,2.0,0,0,1,0,0,1,0,0,0
8929,18,2021,1,1.0,47,64,2.0,0,0,0,0,1,1,0,0,0
8930,18,2021,1,1.0,47,65,1.0,0,0,1,0,0,0,1,0,0


In [15]:
# #Training & Test set 으로 나눠주기
# x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [16]:
result = []
kk = y_train
for k in range(2):
    if k == 1:
        del pd_list[0]['prefer_foot']
        del pd_list[1]['prefer_foot']
        del pd_list[2]['prefer_foot']
        del pd_list[3]['prefer_foot']

    for i in range(4):
        x_train = pd_list[i]
        y_train = kk

        x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train, y_train, test_size=0.2, random_state=0)

        input_dim_num = len(x_train.iloc[i])

        model_origin = models.Sequential(name='model_origin')


        model_origin.add(layers.Dense(input_dim=input_dim_num, units=input_dim_num*2, activation='relu',
                           kernel_initializer = initializers.he_uniform()))

        model_origin.add(layers.Dense(units=input_dim_num*2, activation='relu',
                           kernel_initializer = initializers.he_uniform()))

        model_origin.add(layers.Dense(units=input_dim_num*2, activation='relu',
                           kernel_initializer = initializers.he_uniform()))

        model_origin.add(layers.Dense(units=1, activation=None))


        model_origin.compile(optimizer=optimizers.Adam(),
                  loss=losses.mean_squared_error,
                  metrics=['mean_squared_error']) 

        # model fitting 기본맛
        model_name = model_origin.name

        history_origin = model_origin.fit(x_train, y_train,
                                          batch_size = 100,  #default batch_size = 32
                                          epochs = 1000,
                                          )

        result_origin = model_origin.evaluate(x_test, y_test, batch_size=100)
        result.append(result_origin)

Epoch 1/1000
72/72 [==============================] - 0s 690us/step - loss: 41400406114304.0000 - mean_squared_error: 41400406114304.0000
Epoch 2/1000
72/72 [==============================] - 0s 648us/step - loss: 41302188097536.0000 - mean_squared_error: 41302188097536.0000
Epoch 3/1000
72/72 [==============================] - 0s 634us/step - loss: 40805804802048.0000 - mean_squared_error: 40805804802048.0000
Epoch 4/1000
72/72 [==============================] - 0s 620us/step - loss: 39315493093376.0000 - mean_squared_error: 39315497287680.0000
Epoch 5/1000
72/72 [==============================] - 0s 662us/step - loss: 36669235396608.0000 - mean_squared_error: 36669235396608.0000
Epoch 6/1000
72/72 [==============================] - 0s 662us/step - loss: 34441256763392.0000 - mean_squared_error: 34441256763392.0000
Epoch 7/1000
72/72 [==============================] - 0s 648us/step - loss: 33719503028224.0000 - mean_squared_error: 33719503028224.0000
Epoch 8/1000
72/72 [==============

72/72 [==============================] - 0s 620us/step - loss: 33416435204096.0000 - mean_squared_error: 33416435204096.0000
Epoch 61/1000
72/72 [==============================] - 0s 634us/step - loss: 33419039866880.0000 - mean_squared_error: 33419039866880.0000
Epoch 62/1000
72/72 [==============================] - 0s 634us/step - loss: 33410496069632.0000 - mean_squared_error: 33410496069632.0000
Epoch 63/1000
72/72 [==============================] - 0s 620us/step - loss: 33403369947136.0000 - mean_squared_error: 33403369947136.0000
Epoch 64/1000
72/72 [==============================] - 0s 634us/step - loss: 33398605217792.0000 - mean_squared_error: 33398605217792.0000
Epoch 65/1000
72/72 [==============================] - 0s 620us/step - loss: 33388241092608.0000 - mean_squared_error: 33388241092608.0000
Epoch 66/1000
72/72 [==============================] - 0s 634us/step - loss: 33385938419712.0000 - mean_squared_error: 33385940516864.0000
Epoch 67/1000
72/72 [====================

72/72 [==============================] - 0s 662us/step - loss: 8184603869184.0000 - mean_squared_error: 8184603869184.0000
Epoch 177/1000
72/72 [==============================] - 0s 606us/step - loss: 7877846630400.0000 - mean_squared_error: 7877846630400.0000
Epoch 178/1000
72/72 [==============================] - 0s 606us/step - loss: 7648273498112.0000 - mean_squared_error: 7648273498112.0000
Epoch 179/1000
72/72 [==============================] - 0s 620us/step - loss: 7391175245824.0000 - mean_squared_error: 7391175245824.0000
Epoch 180/1000
72/72 [==============================] - 0s 606us/step - loss: 7204173774848.0000 - mean_squared_error: 7204173774848.0000
Epoch 181/1000
72/72 [==============================] - 0s 606us/step - loss: 6993832050688.0000 - mean_squared_error: 6993832050688.0000
Epoch 182/1000
72/72 [==============================] - 0s 592us/step - loss: 6803769262080.0000 - mean_squared_error: 6803769262080.0000
Epoch 183/1000
72/72 [===========================

72/72 [==============================] - 0s 620us/step - loss: 2427926347776.0000 - mean_squared_error: 2427926085632.0000
Epoch 295/1000
72/72 [==============================] - 0s 662us/step - loss: 2341387894784.0000 - mean_squared_error: 2341387894784.0000
Epoch 296/1000
72/72 [==============================] - 0s 634us/step - loss: 2319862988800.0000 - mean_squared_error: 2319862988800.0000
Epoch 297/1000
72/72 [==============================] - 0s 620us/step - loss: 2335693602816.0000 - mean_squared_error: 2335693602816.0000
Epoch 298/1000
72/72 [==============================] - 0s 620us/step - loss: 2316450922496.0000 - mean_squared_error: 2316450922496.0000
Epoch 299/1000
72/72 [==============================] - 0s 634us/step - loss: 2306077884416.0000 - mean_squared_error: 2306077884416.0000
Epoch 300/1000
72/72 [==============================] - 0s 620us/step - loss: 2300784148480.0000 - mean_squared_error: 2300784148480.0000
Epoch 301/1000
72/72 [===========================

72/72 [==============================] - 0s 634us/step - loss: 1799685406720.0000 - mean_squared_error: 1799685406720.0000
Epoch 413/1000
72/72 [==============================] - 0s 634us/step - loss: 1770419126272.0000 - mean_squared_error: 1770419126272.0000
Epoch 414/1000
72/72 [==============================] - 0s 620us/step - loss: 1769220472832.0000 - mean_squared_error: 1769220472832.0000
Epoch 415/1000
72/72 [==============================] - 0s 620us/step - loss: 1790754029568.0000 - mean_squared_error: 1790754029568.0000
Epoch 416/1000
72/72 [==============================] - 0s 606us/step - loss: 1761281572864.0000 - mean_squared_error: 1761281572864.0000
Epoch 417/1000
72/72 [==============================] - 0s 634us/step - loss: 1785522683904.0000 - mean_squared_error: 1785522683904.0000
Epoch 418/1000
72/72 [==============================] - 0s 606us/step - loss: 1795723231232.0000 - mean_squared_error: 1795723231232.0000
Epoch 419/1000
72/72 [===========================

...

72/72 [==============================] - 0s 690us/step - loss: 1235960594432.0000 - mean_squared_error: 1235960594432.0000
Epoch 547/1000
72/72 [==============================] - 0s 704us/step - loss: 1231467053056.0000 - mean_squared_error: 1231467053056.0000
Epoch 548/1000
72/72 [==============================] - 0s 704us/step - loss: 1277574381568.0000 - mean_squared_error: 1277574381568.0000
Epoch 549/1000
72/72 [==============================] - 0s 690us/step - loss: 1225826631680.0000 - mean_squared_error: 1225826631680.0000
Epoch 550/1000
72/72 [==============================] - 0s 702us/step - loss: 1249435320320.0000 - mean_squared_error: 1249435320320.0000
Epoch 551/1000
72/72 [==============================] - 0s 718us/step - loss: 1201866407936.0000 - mean_squared_error: 1201866407936.0000
Epoch 552/1000
72/72 [==============================] - 0s 733us/step - loss: 1198124826624.0000 - mean_squared_error: 1198124826624.0000
Epoch 553/1000
72/72 [===========================

72/72 [==============================] - 0s 718us/step - loss: 1192095645696.0000 - mean_squared_error: 1192095645696.0000
Epoch 665/1000
72/72 [==============================] - 0s 718us/step - loss: 1207808557056.0000 - mean_squared_error: 1207808557056.0000
Epoch 666/1000
72/72 [==============================] - 0s 690us/step - loss: 1149246636032.0000 - mean_squared_error: 1149246636032.0000
Epoch 667/1000
72/72 [==============================] - 0s 704us/step - loss: 1104569303040.0000 - mean_squared_error: 1104569171968.0000
Epoch 668/1000
72/72 [==============================] - 0s 733us/step - loss: 1147693563904.0000 - mean_squared_error: 1147693563904.0000
Epoch 669/1000
72/72 [==============================] - 0s 733us/step - loss: 1129516236800.0000 - mean_squared_error: 1129516236800.0000
Epoch 670/1000
72/72 [==============================] - 0s 761us/step - loss: 1122308980736.0000 - mean_squared_error: 1122308980736.0000
Epoch 671/1000
72/72 [===========================

72/72 [==============================] - 0s 690us/step - loss: 1167534456832.0000 - mean_squared_error: 1167534456832.0000
Epoch 783/1000
72/72 [==============================] - 0s 733us/step - loss: 1071809888256.0000 - mean_squared_error: 1071809888256.0000
Epoch 784/1000
72/72 [==============================] - 0s 718us/step - loss: 1060211654656.0000 - mean_squared_error: 1060211654656.0000
Epoch 785/1000
72/72 [==============================] - 0s 690us/step - loss: 1113805029376.0000 - mean_squared_error: 1113805029376.0000
Epoch 786/1000
72/72 [==============================] - 0s 690us/step - loss: 1050042105856.0000 - mean_squared_error: 1050042105856.0000
Epoch 787/1000
72/72 [==============================] - 0s 690us/step - loss: 1068184371200.0000 - mean_squared_error: 1068184305664.0000
Epoch 788/1000
72/72 [==============================] - 0s 690us/step - loss: 1121590575104.0000 - mean_squared_error: 1121590575104.0000
Epoch 789/1000
72/72 [===========================

72/72 [==============================] - 0s 718us/step - loss: 1024384696320.0000 - mean_squared_error: 1024384696320.0000
Epoch 901/1000
72/72 [==============================] - 0s 747us/step - loss: 1007496134656.0000 - mean_squared_error: 1007496134656.0000
Epoch 902/1000
72/72 [==============================] - 0s 690us/step - loss: 1053104472064.0000 - mean_squared_error: 1053104472064.0000
Epoch 903/1000
72/72 [==============================] - 0s 690us/step - loss: 970794991616.0000 - mean_squared_error: 970794991616.0000
Epoch 904/1000
72/72 [==============================] - 0s 690us/step - loss: 946968133632.0000 - mean_squared_error: 946968133632.0000
Epoch 905/1000
72/72 [==============================] - 0s 704us/step - loss: 962987032576.0000 - mean_squared_error: 962987032576.0000
Epoch 906/1000
72/72 [==============================] - 0s 690us/step - loss: 973815021568.0000 - mean_squared_error: 973815021568.0000
Epoch 907/1000
72/72 [==============================] - 0

72/72 [==============================] - 0s 704us/step - loss: 991581241344.0000 - mean_squared_error: 991581241344.0000
Epoch 961/1000
72/72 [==============================] - 0s 704us/step - loss: 969019490304.0000 - mean_squared_error: 969019555840.0000
Epoch 962/1000
72/72 [==============================] - 0s 704us/step - loss: 1043641729024.0000 - mean_squared_error: 1043641860096.0000
Epoch 963/1000
72/72 [==============================] - 0s 676us/step - loss: 990865063936.0000 - mean_squared_error: 990864932864.0000
Epoch 964/1000
72/72 [==============================] - 0s 690us/step - loss: 950808084480.0000 - mean_squared_error: 950808084480.0000
Epoch 965/1000
72/72 [==============================] - 0s 690us/step - loss: 963958472704.0000 - mean_squared_error: 963958472704.0000
Epoch 966/1000
72/72 [==============================] - 0s 690us/step - loss: 933939576832.0000 - mean_squared_error: 933939576832.0000
Epoch 967/1000
72/72 [==============================] - 0s 67

In [17]:
result

[[1186302787584.0, 1186302918656.0],
 [1313974255616.0, 1313974255616.0],
 [920094507008.0, 920094441472.0],
 [1224613167104.0, 1224613167104.0],
 [1299093520384.0, 1299093520384.0],
 [1344199196672.0, 1344199196672.0],
 [1136777625600.0, 1136777625600.0],
 [1233793318912.0, 1233793318912.0]]

In [14]:
input_dim_num = len(x_train.iloc[0])

In [15]:
#Sequential
model_origin = models.Sequential(name='model_origin')


model_origin.add(layers.Dense(input_dim=input_dim_num, units=input_dim_num*2, activation='relu',
                       kernel_initializer = initializers.he_uniform()))

model_origin.add(layers.Dense(units=input_dim_num*2, activation='relu',
                       kernel_initializer = initializers.he_uniform()))

model_origin.add(layers.Dense(units=input_dim_num*2, activation='relu',
                       kernel_initializer = initializers.he_uniform()))

model_origin.add(layers.Dense(units=1, activation=None))


model_origin.compile(optimizer=optimizers.Adam(),
              loss=losses.mean_squared_error,
              metrics=['mean_squared_error']) 

# summary
model_origin.summary()

Model: "model_origin"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
_________________________________________________________________


In [16]:
# model fitting 기본맛
model_name = model_origin.name

#make check point
callback_check = tf.keras.callbacks.ModelCheckpoint( 
    filepath = '02_predictive_data/checkpoint/'+ model_name + '.h5', # 모델 저장 경로, ex) '{epoch:02d}-{val_loss:.5f}.h5'
    monitor = 'loss', # 'loss', 'val_loss', 'accuracy', etc.
    verbose = 0, # 0 or 1
    save_best_only = False, # True 오차가 줄어줄때 마다 저장
    save_weights_only = False, # True 오차가 적은 가중치를 저장
    mode = 'auto', # 'auto', 'min', 'max'
    save_freq = 'epoch', # 'epoch' or integer(== # of batches) 
    #save_freq = 5 * batch_size # == saves the model's weights every 5 epochs (variable 'batch_size' should be set already)
)


history_origin = model_origin.fit(x_train, y_train,
                                  batch_size = 100,  #default batch_size = 32
                                  epochs = 1000,
                                  callbacks = [callback_check])

Epoch 1/1000
72/72 [==============================] - 0s 690us/step - loss: 41397105197056.0000 - mean_squared_error: 41397109391360.0000
Epoch 2/1000
72/72 [==============================] - 0s 690us/step - loss: 41294957117440.0000 - mean_squared_error: 41294957117440.0000
Epoch 3/1000
72/72 [==============================] - 0s 733us/step - loss: 40829678780416.0000 - mean_squared_error: 40829678780416.0000
Epoch 4/1000
72/72 [==============================] - 0s 775us/step - loss: 39504429711360.0000 - mean_squared_error: 39504429711360.0000
Epoch 5/1000
72/72 [==============================] - 0s 718us/step - loss: 37083548745728.0000 - mean_squared_error: 37083548745728.0000
Epoch 6/1000
72/72 [==============================] - 0s 718us/step - loss: 34681368084480.0000 - mean_squared_error: 34681368084480.0000
Epoch 7/1000
72/72 [==============================] - 0s 704us/step - loss: 33759493619712.0000 - mean_squared_error: 33759493619712.0000
Epoch 8/1000
72/72 [==============

72/72 [==============================] - 0s 662us/step - loss: 33395830685696.0000 - mean_squared_error: 33395830685696.0000
Epoch 61/1000
72/72 [==============================] - 0s 634us/step - loss: 33383203733504.0000 - mean_squared_error: 33383203733504.0000
Epoch 62/1000
72/72 [==============================] - 0s 648us/step - loss: 33379781181440.0000 - mean_squared_error: 33379781181440.0000
Epoch 63/1000
72/72 [==============================] - 0s 676us/step - loss: 33372898328576.0000 - mean_squared_error: 33372900425728.0000
Epoch 64/1000
72/72 [==============================] - 0s 648us/step - loss: 33365656862720.0000 - mean_squared_error: 33365656862720.0000
Epoch 65/1000
72/72 [==============================] - 0s 662us/step - loss: 33358448951296.0000 - mean_squared_error: 33358448951296.0000
Epoch 66/1000
72/72 [==============================] - 0s 662us/step - loss: 33361475141632.0000 - mean_squared_error: 33361473044480.0000
Epoch 67/1000
72/72 [====================

72/72 [==============================] - 0s 676us/step - loss: 7001883017216.0000 - mean_squared_error: 7001883017216.0000
Epoch 177/1000
72/72 [==============================] - 0s 699us/step - loss: 6795511726080.0000 - mean_squared_error: 6795511726080.0000
Epoch 178/1000
72/72 [==============================] - 0s 733us/step - loss: 6645070954496.0000 - mean_squared_error: 6645070954496.0000
Epoch 179/1000
72/72 [==============================] - 0s 733us/step - loss: 6437671534592.0000 - mean_squared_error: 6437671534592.0000
Epoch 180/1000
72/72 [==============================] - 0s 676us/step - loss: 6326457991168.0000 - mean_squared_error: 6326457991168.0000
Epoch 181/1000
72/72 [==============================] - 0s 690us/step - loss: 6165566062592.0000 - mean_squared_error: 6165566062592.0000
Epoch 182/1000
72/72 [==============================] - 0s 676us/step - loss: 6017146945536.0000 - mean_squared_error: 6017146945536.0000
Epoch 183/1000
72/72 [===========================

72/72 [==============================] - 0s 690us/step - loss: 2469867552768.0000 - mean_squared_error: 2469867552768.0000
Epoch 295/1000
72/72 [==============================] - 0s 733us/step - loss: 2436783669248.0000 - mean_squared_error: 2436783669248.0000
Epoch 296/1000
72/72 [==============================] - 0s 704us/step - loss: 2436620353536.0000 - mean_squared_error: 2436620353536.0000
Epoch 297/1000
72/72 [==============================] - 0s 690us/step - loss: 2431304859648.0000 - mean_squared_error: 2431304859648.0000
Epoch 298/1000
72/72 [==============================] - 0s 662us/step - loss: 2459714715648.0000 - mean_squared_error: 2459714715648.0000
Epoch 299/1000
72/72 [==============================] - 0s 662us/step - loss: 2411800035328.0000 - mean_squared_error: 2411800035328.0000
Epoch 300/1000
72/72 [==============================] - 0s 662us/step - loss: 2391787700224.0000 - mean_squared_error: 2391787700224.0000
Epoch 301/1000
72/72 [===========================

72/72 [==============================] - 0s 648us/step - loss: 1923277914112.0000 - mean_squared_error: 1923277914112.0000
Epoch 413/1000
72/72 [==============================] - 0s 662us/step - loss: 1899054759936.0000 - mean_squared_error: 1899054759936.0000
Epoch 414/1000
72/72 [==============================] - 0s 676us/step - loss: 1910079094784.0000 - mean_squared_error: 1910079094784.0000
Epoch 415/1000
72/72 [==============================] - 0s 662us/step - loss: 1885865115648.0000 - mean_squared_error: 1885865115648.0000
Epoch 416/1000
72/72 [==============================] - 0s 648us/step - loss: 1940051722240.0000 - mean_squared_error: 1940051722240.0000
Epoch 417/1000
72/72 [==============================] - 0s 648us/step - loss: 1863500955648.0000 - mean_squared_error: 1863500955648.0000
Epoch 418/1000
72/72 [==============================] - 0s 648us/step - loss: 1901559939072.0000 - mean_squared_error: 1901559939072.0000
Epoch 419/1000
72/72 [===========================

72/72 [==============================] - 0s 718us/step - loss: 1781842182144.0000 - mean_squared_error: 1781842182144.0000
Epoch 531/1000
72/72 [==============================] - 0s 662us/step - loss: 1618075189248.0000 - mean_squared_error: 1618075189248.0000
Epoch 532/1000
72/72 [==============================] - 0s 648us/step - loss: 1637718556672.0000 - mean_squared_error: 1637718556672.0000
Epoch 533/1000
72/72 [==============================] - 0s 676us/step - loss: 1634385920000.0000 - mean_squared_error: 1634385920000.0000
Epoch 534/1000
72/72 [==============================] - 0s 648us/step - loss: 1652579500032.0000 - mean_squared_error: 1652579500032.0000
Epoch 535/1000
72/72 [==============================] - 0s 690us/step - loss: 1622349971456.0000 - mean_squared_error: 1622349971456.0000
Epoch 536/1000
72/72 [==============================] - 0s 761us/step - loss: 1608354889728.0000 - mean_squared_error: 1608354889728.0000
Epoch 537/1000
72/72 [===========================

72/72 [==============================] - 0s 676us/step - loss: 1483432525824.0000 - mean_squared_error: 1483432525824.0000
Epoch 649/1000
72/72 [==============================] - 0s 690us/step - loss: 1469849272320.0000 - mean_squared_error: 1469849272320.0000
Epoch 650/1000
72/72 [==============================] - 0s 634us/step - loss: 1455638315008.0000 - mean_squared_error: 1455638315008.0000
Epoch 651/1000
72/72 [==============================] - 0s 662us/step - loss: 1450071162880.0000 - mean_squared_error: 1450071162880.0000
Epoch 652/1000
72/72 [==============================] - 0s 690us/step - loss: 1440169197568.0000 - mean_squared_error: 1440169197568.0000
Epoch 653/1000
72/72 [==============================] - 0s 662us/step - loss: 1421204652032.0000 - mean_squared_error: 1421204652032.0000
Epoch 654/1000
72/72 [==============================] - 0s 747us/step - loss: 1478293716992.0000 - mean_squared_error: 1478293716992.0000
Epoch 655/1000
72/72 [===========================

72/72 [==============================] - 0s 671us/step - loss: 1367009525760.0000 - mean_squared_error: 1367009525760.0000
Epoch 767/1000
72/72 [==============================] - 0s 641us/step - loss: 1314802368512.0000 - mean_squared_error: 1314802368512.0000
Epoch 768/1000
72/72 [==============================] - 0s 662us/step - loss: 1370372571136.0000 - mean_squared_error: 1370372571136.0000
Epoch 769/1000
72/72 [==============================] - 0s 690us/step - loss: 1327389999104.0000 - mean_squared_error: 1327389999104.0000
Epoch 770/1000
72/72 [==============================] - 0s 648us/step - loss: 1330984910848.0000 - mean_squared_error: 1330984910848.0000
Epoch 771/1000
72/72 [==============================] - 0s 676us/step - loss: 1385246097408.0000 - mean_squared_error: 1385246097408.0000
Epoch 772/1000
72/72 [==============================] - 0s 648us/step - loss: 1366605955072.0000 - mean_squared_error: 1366605955072.0000
Epoch 773/1000
72/72 [===========================

72/72 [==============================] - 0s 648us/step - loss: 1274867089408.0000 - mean_squared_error: 1274867089408.0000
Epoch 885/1000
72/72 [==============================] - 0s 648us/step - loss: 1267236601856.0000 - mean_squared_error: 1267236601856.0000
Epoch 886/1000
72/72 [==============================] - 0s 676us/step - loss: 1271993597952.0000 - mean_squared_error: 1271993729024.0000
Epoch 887/1000
72/72 [==============================] - 0s 676us/step - loss: 1315505045504.0000 - mean_squared_error: 1315505045504.0000
Epoch 888/1000
72/72 [==============================] - 0s 648us/step - loss: 1276039921664.0000 - mean_squared_error: 1276039921664.0000
Epoch 889/1000
72/72 [==============================] - 0s 648us/step - loss: 1244253126656.0000 - mean_squared_error: 1244253126656.0000
Epoch 890/1000
72/72 [==============================] - 0s 648us/step - loss: 1288429371392.0000 - mean_squared_error: 1288429371392.0000
Epoch 891/1000
72/72 [===========================

In [17]:
result_origin = model_origin.evaluate(x_test, y_test, batch_size=100)

18/18 [==============================] - 0s 647us/step - loss: 1272699682816.0000 - mean_squared_error: 1272699551744.0000


In [ ]:

1272699682816.0000 
1163257839616.0000 
978505433088.0000
1080998952960.0000 

In [70]:
#Sequential 배치적용 
model_batch = models.Sequential(name = 'model_batch')

# Regression

model_batch.add(layers.Dense(input_dim=input_dim_num, units=input_dim_num*2, activation=None,
                       kernel_initializer = initializers.he_uniform()))
model_batch.add(layers.BatchNormalization())
model_batch.add(layers.Activation('relu'))


model_batch.add(layers.Dense(units=input_dim_num*2, activation=None,
                       kernel_initializer = initializers.he_uniform()))
model_batch.add(layers.BatchNormalization())
model_batch.add(layers.Activation('relu'))


model_batch.add(layers.Dense(units=input_dim_num*2, activation=None,
                       kernel_initializer = initializers.he_uniform()))
model_batch.add(layers.BatchNormalization())
model_batch.add(layers.Activation('relu'))


model_batch.add(layers.Dense(units=1, activation=None))

model_batch.compile(optimizer=optimizers.Adam(),
              loss=losses.mean_squared_error,
              metrics=['mean_squared_error']) 

# summary
model_batch.summary()

Model: "model_batch"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 30)                480       
_________________________________________________________________
batch_normalization_3 (Batch (None, 30)                120       
_________________________________________________________________
activation_3 (Activation)    (None, 30)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 30)                930       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30)                120       
_________________________________________________________________
activation_4 (Activation)    (None, 30)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 30)                

In [71]:
# model fitting bachnomal
model_name = model_batch.name

#make check point
callback_check = tf.keras.callbacks.ModelCheckpoint( 
    filepath = '02_predictive_data/checkpoint/'+ model_name + '.h5', # 모델 저장 경로, ex) '{epoch:02d}-{val_loss:.5f}.h5'
    monitor = 'loss', # 'loss', 'val_loss', 'accuracy', etc.
    verbose = 0, # 0 or 1
    save_best_only = False, # True 오차가 줄어줄때 마다 저장
    save_weights_only = False, # True 오차가 적은 가중치를 저장
    mode = 'auto', # 'auto', 'min', 'max'
    save_freq = 'epoch', # 'epoch' or integer(== # of batches) 
    #save_freq = 5 * batch_size # == saves the model's weights every 5 epochs (variable 'batch_size' should be set already)
)

history_batch = model_batch.fit(x_train, y_train,
                                batch_size = 100,  #default batch_size = 32
                                epochs = 1000,
                                callbacks = [callback_check])

Epoch 1/1000
72/72 [==============================] - 1s 1ms/step - loss: 41393684.0000 - mean_squared_error: 41393684.0000
Epoch 2/1000
72/72 [==============================] - 0s 1ms/step - loss: 41362216.0000 - mean_squared_error: 41362216.0000
Epoch 3/1000
72/72 [==============================] - 0s 1ms/step - loss: 41332348.0000 - mean_squared_error: 41332348.0000
Epoch 4/1000
72/72 [==============================] - 0s 1ms/step - loss: 41295824.0000 - mean_squared_error: 41295824.0000
Epoch 5/1000
72/72 [==============================] - 0s 1ms/step - loss: 41245528.0000 - mean_squared_error: 41245528.0000
Epoch 6/1000
72/72 [==============================] - 0s 1ms/step - loss: 41195836.0000 - mean_squared_error: 41195836.0000
Epoch 7/1000
72/72 [==============================] - 0s 1ms/step - loss: 41135776.0000 - mean_squared_error: 41135776.0000
Epoch 8/1000
72/72 [==============================] - 0s 1ms/step - loss: 41072148.0000 - mean_squared_error: 41072148.0000
Epoch 9/

72/72 [==============================] - 0s 1ms/step - loss: 10158019.0000 - mean_squared_error: 10158019.0000
Epoch 132/1000
72/72 [==============================] - 0s 1ms/step - loss: 10268843.0000 - mean_squared_error: 10268843.0000
Epoch 133/1000
72/72 [==============================] - 0s 1ms/step - loss: 10089877.0000 - mean_squared_error: 10089877.0000
Epoch 134/1000
72/72 [==============================] - 0s 1ms/step - loss: 10052275.0000 - mean_squared_error: 10052275.0000
Epoch 135/1000
72/72 [==============================] - 0s 1ms/step - loss: 9765454.0000 - mean_squared_error: 9765454.0000
Epoch 136/1000
72/72 [==============================] - 0s 1ms/step - loss: 9509713.0000 - mean_squared_error: 9509713.0000
Epoch 137/1000
72/72 [==============================] - 0s 1ms/step - loss: 9792429.0000 - mean_squared_error: 9792429.0000
Epoch 138/1000
72/72 [==============================] - 0s 1ms/step - loss: 9104147.0000 - mean_squared_error: 9104147.0000
Epoch 139/1000


72/72 [==============================] - 0s 1ms/step - loss: 3500188.5000 - mean_squared_error: 3500188.5000
Epoch 264/1000
72/72 [==============================] - 0s 1ms/step - loss: 3074314.5000 - mean_squared_error: 3074314.5000
Epoch 265/1000
72/72 [==============================] - 0s 1ms/step - loss: 3316885.5000 - mean_squared_error: 3316885.7500
Epoch 266/1000
72/72 [==============================] - 0s 1ms/step - loss: 3119955.0000 - mean_squared_error: 3119955.0000
Epoch 267/1000
72/72 [==============================] - 0s 1ms/step - loss: 2981655.2500 - mean_squared_error: 2981655.2500
Epoch 268/1000
72/72 [==============================] - 0s 1ms/step - loss: 2822774.2500 - mean_squared_error: 2822774.2500
Epoch 269/1000
72/72 [==============================] - 0s 1ms/step - loss: 2871643.0000 - mean_squared_error: 2871643.0000
Epoch 270/1000
72/72 [==============================] - 0s 1ms/step - loss: 2473622.0000 - mean_squared_error: 2473622.0000
Epoch 271/1000
72/72 [=

72/72 [==============================] - 0s 1ms/step - loss: 2408124.0000 - mean_squared_error: 2408124.0000
Epoch 396/1000
72/72 [==============================] - 0s 1ms/step - loss: 2403215.0000 - mean_squared_error: 2403215.0000
Epoch 397/1000
72/72 [==============================] - 0s 1ms/step - loss: 1740728.6250 - mean_squared_error: 1740728.6250
Epoch 398/1000
72/72 [==============================] - 0s 1ms/step - loss: 1937913.5000 - mean_squared_error: 1937913.5000
Epoch 399/1000
72/72 [==============================] - 0s 1ms/step - loss: 2029091.1250 - mean_squared_error: 2029091.1250
Epoch 400/1000
72/72 [==============================] - 0s 1ms/step - loss: 1857794.6250 - mean_squared_error: 1857794.6250
Epoch 401/1000
72/72 [==============================] - 0s 1ms/step - loss: 1891953.3750 - mean_squared_error: 1891953.3750
Epoch 402/1000
72/72 [==============================] - 0s 1ms/step - loss: 2044305.3750 - mean_squared_error: 2044305.3750
Epoch 403/1000
72/72 [=

72/72 [==============================] - 0s 2ms/step - loss: 1531474.0000 - mean_squared_error: 1531474.2500
Epoch 528/1000
72/72 [==============================] - 0s 1ms/step - loss: 1833832.6250 - mean_squared_error: 1833832.6250
Epoch 529/1000
72/72 [==============================] - 0s 1ms/step - loss: 1452424.1250 - mean_squared_error: 1452424.1250
Epoch 530/1000
72/72 [==============================] - 0s 1ms/step - loss: 1224443.7500 - mean_squared_error: 1224443.7500
Epoch 531/1000
72/72 [==============================] - 0s 1ms/step - loss: 1567694.7500 - mean_squared_error: 1567694.7500
Epoch 532/1000
72/72 [==============================] - 0s 1ms/step - loss: 1451251.7500 - mean_squared_error: 1451251.7500
Epoch 533/1000
72/72 [==============================] - 0s 1ms/step - loss: 1408854.3750 - mean_squared_error: 1408854.1250
Epoch 534/1000
72/72 [==============================] - 0s 1ms/step - loss: 1277075.3750 - mean_squared_error: 1277075.3750
Epoch 535/1000
72/72 [=

72/72 [==============================] - 0s 1ms/step - loss: 1324689.5000 - mean_squared_error: 1324689.5000
Epoch 660/1000
72/72 [==============================] - 0s 1ms/step - loss: 1333648.0000 - mean_squared_error: 1333648.0000
Epoch 661/1000
72/72 [==============================] - 0s 1ms/step - loss: 1278021.3750 - mean_squared_error: 1278021.3750
Epoch 662/1000
72/72 [==============================] - 0s 1ms/step - loss: 1027476.8125 - mean_squared_error: 1027476.7500
Epoch 663/1000
72/72 [==============================] - 0s 1ms/step - loss: 1164475.3750 - mean_squared_error: 1164475.3750
Epoch 664/1000
72/72 [==============================] - 0s 1ms/step - loss: 1076364.3750 - mean_squared_error: 1076364.3750
Epoch 665/1000
72/72 [==============================] - 0s 1ms/step - loss: 1256915.7500 - mean_squared_error: 1256915.6250
Epoch 666/1000
72/72 [==============================] - 0s 1ms/step - loss: 1237956.1250 - mean_squared_error: 1237956.1250
Epoch 667/1000
72/72 [=

72/72 [==============================] - 0s 1ms/step - loss: 1014851.5625 - mean_squared_error: 1014851.5625
Epoch 792/1000
72/72 [==============================] - 0s 1ms/step - loss: 996201.9375 - mean_squared_error: 996201.9375
Epoch 793/1000
72/72 [==============================] - 0s 1ms/step - loss: 1119458.0000 - mean_squared_error: 1119458.0000
Epoch 794/1000
72/72 [==============================] - 0s 1ms/step - loss: 1204897.6250 - mean_squared_error: 1204897.6250
Epoch 795/1000
72/72 [==============================] - 0s 1ms/step - loss: 1224163.3750 - mean_squared_error: 1224163.3750
Epoch 796/1000
72/72 [==============================] - 0s 1ms/step - loss: 1062526.0000 - mean_squared_error: 1062526.0000
Epoch 797/1000
72/72 [==============================] - 0s 1ms/step - loss: 1107971.1250 - mean_squared_error: 1107971.1250
Epoch 798/1000
72/72 [==============================] - 0s 1ms/step - loss: 1075227.0000 - mean_squared_error: 1075227.0000
Epoch 799/1000
72/72 [===

72/72 [==============================] - 0s 1ms/step - loss: 849654.5625 - mean_squared_error: 849654.5625
Epoch 924/1000
72/72 [==============================] - 0s 1ms/step - loss: 873780.8750 - mean_squared_error: 873780.8750
Epoch 925/1000
72/72 [==============================] - 0s 1ms/step - loss: 933837.7500 - mean_squared_error: 933837.7500
Epoch 926/1000
72/72 [==============================] - 0s 1ms/step - loss: 984730.8125 - mean_squared_error: 984730.8125
Epoch 927/1000
72/72 [==============================] - 0s 1ms/step - loss: 924634.5000 - mean_squared_error: 924634.5000
Epoch 928/1000
72/72 [==============================] - 0s 1ms/step - loss: 976717.6250 - mean_squared_error: 976717.6250
Epoch 929/1000
72/72 [==============================] - 0s 1ms/step - loss: 956365.7500 - mean_squared_error: 956365.9375
Epoch 930/1000
72/72 [==============================] - 0s 1ms/step - loss: 1053357.8750 - mean_squared_error: 1053357.8750
Epoch 931/1000
72/72 [===============

In [72]:
#Sequential 드롭아웃 적용
model_drop = models.Sequential(name = 'model_drop')

model_drop.add(layers.Dense(input_dim=input_dim_num, units=input_dim_num*2, activation='relu',
                       kernel_initializer = initializers.he_uniform()))

model_drop.add(layers.Dense(units=input_dim_num*2, activation='relu',
                       kernel_initializer = initializers.he_uniform()))

model_drop.add(layers.Dense(units=input_dim_num*2, activation='relu',
                       kernel_initializer = initializers.he_uniform()))
model_drop.add(layers.Dropout(rate=0.2))

model_drop.add(layers.Dense(units=1, activation=None))


model_drop.compile(optimizer=optimizers.Adam(),
              loss=losses.mean_squared_error,
              metrics=['mean_squared_error']) 

# summary
model_drop.summary()

Model: "model_drop"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 30)                480       
_________________________________________________________________
dense_21 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_22 (Dense)             (None, 30)                930       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 31        
Total params: 2,371
Trainable params: 2,371
Non-trainable params: 0
_________________________________________________________________


In [73]:
# model3 fitting on drop out
model_name = model_drop.name

#make check point
callback_check = tf.keras.callbacks.ModelCheckpoint( 
    filepath = '02_predictive_data/checkpoint/'+ model_name + '.h5', # 모델 저장 경로, ex) '{epoch:02d}-{val_loss:.5f}.h5'
    monitor = 'loss', # 'loss', 'val_loss', 'accuracy', etc.
    verbose = 0, # 0 or 1
    save_best_only = False, # True 오차가 줄어줄때 마다 저장
    save_weights_only = False, # True 오차가 적은 가중치를 저장
    mode = 'auto', # 'auto', 'min', 'max'
    save_freq = 'epoch', # 'epoch' or integer(== # of batches) 
    #save_freq = 5 * batch_size # == saves the model's weights every 5 epochs (variable 'batch_size' should be set already)
)


history_drop = model_drop.fit(x_train, y_train,
                              batch_size = 100,  #default batch_size = 32
                              epochs = 1000,
                              callbacks = [callback_check])

Epoch 1/1000
72/72 [==============================] - 0s 789us/step - loss: 34754380.0000 - mean_squared_error: 34754380.0000
Epoch 2/1000
72/72 [==============================] - 0s 775us/step - loss: 34167476.0000 - mean_squared_error: 34167476.0000
Epoch 3/1000
72/72 [==============================] - 0s 761us/step - loss: 34006180.0000 - mean_squared_error: 34006180.0000
Epoch 4/1000
72/72 [==============================] - 0s 775us/step - loss: 34084080.0000 - mean_squared_error: 34084080.0000
Epoch 5/1000
72/72 [==============================] - 0s 817us/step - loss: 34006220.0000 - mean_squared_error: 34006220.0000
Epoch 6/1000
72/72 [==============================] - 0s 831us/step - loss: 33902428.0000 - mean_squared_error: 33902420.0000
Epoch 7/1000
72/72 [==============================] - 0s 803us/step - loss: 33872480.0000 - mean_squared_error: 33872480.0000
Epoch 8/1000
72/72 [==============================] - 0s 817us/step - loss: 34003756.0000 - mean_squared_error: 340037

72/72 [==============================] - 0s 831us/step - loss: 2951156.7500 - mean_squared_error: 2951156.7500
Epoch 131/1000
72/72 [==============================] - 0s 829us/step - loss: 2686173.2500 - mean_squared_error: 2686173.2500
Epoch 132/1000
72/72 [==============================] - 0s 789us/step - loss: 2808226.2500 - mean_squared_error: 2808226.2500
Epoch 133/1000
72/72 [==============================] - 0s 775us/step - loss: 3054010.2500 - mean_squared_error: 3054010.2500
Epoch 134/1000
72/72 [==============================] - 0s 775us/step - loss: 3040705.7500 - mean_squared_error: 3040705.7500
Epoch 135/1000
72/72 [==============================] - 0s 775us/step - loss: 2530566.7500 - mean_squared_error: 2530566.7500
Epoch 136/1000
72/72 [==============================] - 0s 826us/step - loss: 3194291.0000 - mean_squared_error: 3194291.0000
Epoch 137/1000
72/72 [==============================] - 0s 824us/step - loss: 3615614.5000 - mean_squared_error: 3615614.5000
Epoch 1

72/72 [==============================] - 0s 747us/step - loss: 2760216.2500 - mean_squared_error: 2760216.2500
Epoch 261/1000
72/72 [==============================] - 0s 775us/step - loss: 2574805.7500 - mean_squared_error: 2574805.7500
Epoch 262/1000
72/72 [==============================] - 0s 775us/step - loss: 2196025.0000 - mean_squared_error: 2196025.0000
Epoch 263/1000
72/72 [==============================] - 0s 733us/step - loss: 2218595.7500 - mean_squared_error: 2218595.5000
Epoch 264/1000
72/72 [==============================] - 0s 761us/step - loss: 2702650.7500 - mean_squared_error: 2702650.7500
Epoch 265/1000
72/72 [==============================] - 0s 761us/step - loss: 2792295.0000 - mean_squared_error: 2792295.0000
Epoch 266/1000
72/72 [==============================] - 0s 761us/step - loss: 2526594.5000 - mean_squared_error: 2526594.5000
Epoch 267/1000
72/72 [==============================] - 0s 761us/step - loss: 2571409.5000 - mean_squared_error: 2571409.5000
Epoch 2

72/72 [==============================] - 0s 755us/step - loss: 2216916.0000 - mean_squared_error: 2216916.0000
Epoch 391/1000
72/72 [==============================] - 0s 754us/step - loss: 2523948.5000 - mean_squared_error: 2523948.5000
Epoch 392/1000
72/72 [==============================] - 0s 775us/step - loss: 2688633.0000 - mean_squared_error: 2688633.0000
Epoch 393/1000
72/72 [==============================] - 0s 789us/step - loss: 2364932.7500 - mean_squared_error: 2364932.7500
Epoch 394/1000
72/72 [==============================] - 0s 747us/step - loss: 2222389.2500 - mean_squared_error: 2222389.2500
Epoch 395/1000
72/72 [==============================] - 0s 761us/step - loss: 2006105.6250 - mean_squared_error: 2006105.3750
Epoch 396/1000
72/72 [==============================] - 0s 789us/step - loss: 2639473.7500 - mean_squared_error: 2639473.7500
Epoch 397/1000
72/72 [==============================] - 0s 803us/step - loss: 2832822.5000 - mean_squared_error: 2832822.5000
Epoch 3

72/72 [==============================] - 0s 789us/step - loss: 2365706.7500 - mean_squared_error: 2365706.7500
Epoch 521/1000
72/72 [==============================] - 0s 810us/step - loss: 1881127.5000 - mean_squared_error: 1881127.5000
Epoch 522/1000
72/72 [==============================] - 0s 747us/step - loss: 2279073.2500 - mean_squared_error: 2279073.2500
Epoch 523/1000
72/72 [==============================] - 0s 817us/step - loss: 2800754.5000 - mean_squared_error: 2800754.5000
Epoch 524/1000
72/72 [==============================] - 0s 789us/step - loss: 2006097.5000 - mean_squared_error: 2006097.5000
Epoch 525/1000
72/72 [==============================] - 0s 775us/step - loss: 2301026.7500 - mean_squared_error: 2301027.0000
Epoch 526/1000
72/72 [==============================] - 0s 761us/step - loss: 2404144.0000 - mean_squared_error: 2404144.0000
Epoch 527/1000
72/72 [==============================] - 0s 789us/step - loss: 2174803.2500 - mean_squared_error: 2174803.2500
Epoch 5

72/72 [==============================] - 0s 775us/step - loss: 1870980.6250 - mean_squared_error: 1870980.6250
Epoch 651/1000
72/72 [==============================] - 0s 747us/step - loss: 2373542.5000 - mean_squared_error: 2373542.7500
Epoch 652/1000
72/72 [==============================] - 0s 761us/step - loss: 1747180.7500 - mean_squared_error: 1747180.7500
Epoch 653/1000
72/72 [==============================] - 0s 747us/step - loss: 2129396.0000 - mean_squared_error: 2129396.0000
Epoch 654/1000
72/72 [==============================] - 0s 761us/step - loss: 1797656.7500 - mean_squared_error: 1797656.7500
Epoch 655/1000
72/72 [==============================] - 0s 775us/step - loss: 1911179.1250 - mean_squared_error: 1911179.1250
Epoch 656/1000
72/72 [==============================] - 0s 761us/step - loss: 1862559.0000 - mean_squared_error: 1862559.0000
Epoch 657/1000
72/72 [==============================] - 0s 761us/step - loss: 1904449.6250 - mean_squared_error: 1904449.6250
Epoch 6

72/72 [==============================] - 0s 761us/step - loss: 1602246.6250 - mean_squared_error: 1602246.6250
Epoch 781/1000
72/72 [==============================] - 0s 761us/step - loss: 1703083.2500 - mean_squared_error: 1703083.2500
Epoch 782/1000
72/72 [==============================] - 0s 747us/step - loss: 1757616.5000 - mean_squared_error: 1757616.5000
Epoch 783/1000
72/72 [==============================] - 0s 755us/step - loss: 1839179.3750 - mean_squared_error: 1839179.3750
Epoch 784/1000
72/72 [==============================] - 0s 775us/step - loss: 1786331.3750 - mean_squared_error: 1786331.3750
Epoch 785/1000
72/72 [==============================] - 0s 771us/step - loss: 1763259.1250 - mean_squared_error: 1763259.1250
Epoch 786/1000
72/72 [==============================] - 0s 747us/step - loss: 1816519.1250 - mean_squared_error: 1816519.2500
Epoch 787/1000
72/72 [==============================] - 0s 761us/step - loss: 1652946.6250 - mean_squared_error: 1652946.6250
Epoch 7

72/72 [==============================] - 0s 845us/step - loss: 1838502.5000 - mean_squared_error: 1838502.5000
Epoch 911/1000
72/72 [==============================] - 0s 761us/step - loss: 2174539.0000 - mean_squared_error: 2174539.0000
Epoch 912/1000
72/72 [==============================] - 0s 747us/step - loss: 1833041.1250 - mean_squared_error: 1833041.1250
Epoch 913/1000
72/72 [==============================] - 0s 761us/step - loss: 1602226.3750 - mean_squared_error: 1602226.3750
Epoch 914/1000
72/72 [==============================] - 0s 761us/step - loss: 1837669.1250 - mean_squared_error: 1837669.1250
Epoch 915/1000
72/72 [==============================] - 0s 761us/step - loss: 1856526.7500 - mean_squared_error: 1856526.7500
Epoch 916/1000
72/72 [==============================] - 0s 761us/step - loss: 1655597.3750 - mean_squared_error: 1655597.3750
Epoch 917/1000
72/72 [==============================] - 0s 789us/step - loss: 1939224.8750 - mean_squared_error: 1939224.8750
Epoch 9

In [77]:
result_test_origin = model_origin.evaluate(x_test, y_test, batch_size=100)
result_test_batch = model_batch.evaluate(x_test, y_test, batch_size=100)
result_test_drop = model_drop.evaluate(x_test, y_test, batch_size=100)

18/18 [==============================] - 0s 530us/step - loss: 1317674.3750 - mean_squared_error: 1317674.3750


In [18]:
model1_rmse = np.sqrt(history_origin.history['loss'])
model2_rmse = np.sqrt(history_batch.history['loss'])
model3_rmse = np.sqrt(history_drop.history['loss'])


x_len = np.arange(len(model1_mse))

plt.plot(x_len[:1000], model1_rmse[:1000], marker = '.', c='blue', label = 'model_origin')
plt.plot(x_len[:1000], model2_rmse[:1000], marker = '.', c='red', label = 'model_batch)')
plt.plot(x_len[:1000], model3_rmse[:1000], marker = '.', c='yellow', label = 'model_drop')

plt.legend(loc = 'upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.show()

NameError: name 'history_batch' is not defined

In [101]:
a ='''
성능지표
origin : {}
batch : {}
drop : {}
'''.format(str(result_test_origin[0]), str(result_test_batch[0]), str(result_test_drop[0]))

print(a)


성능지표
origin : 949635.25
batch : 19913259008.0
drop : 1317674.375

